In [1]:
import torch
import transformers
import datasets
import huggingface_hub
import safetensors

In [2]:
cache_dir = '/Users/christopher/Documents/unirepsCache'
# cache_dir = '/net/scratch2/chriswolfram/hf_cache'

In [3]:
huggingface_hub.login(new_session=False)

In [4]:
model_name = 'meta-llama/Llama-3.2-1B'

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, device_map='auto', cache_dir=cache_dir)
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, torch_dtype='auto', device_map='auto', cache_dir=cache_dir)

In [6]:
# Add padding token if needed
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [1]:
# TODO: Use split slicing and maybe shuffling+streaming to extract samples from large datasets

In [52]:
dataset = datasets.load_datadataset('stanfordnlp/imdb', cache_dir=cache_dir)
dataset = dataset['test'].take(9)

In [8]:
model.get_output_embeddings().weight.detach()

tensor([[ 0.0045,  0.0166,  0.0210,  ..., -0.0054, -0.0422, -0.0315],
        [ 0.0215, -0.0238,  0.0211,  ..., -0.0107, -0.0011, -0.0374],
        [ 0.0136,  0.0104,  0.0128,  ...,  0.0081, -0.0122,  0.0051],
        ...,
        [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066],
        [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066],
        [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066]],
       device='cuda:0', dtype=torch.bfloat16)

In [ ]:
def compute_embeddings(examples):
    tokens = tokenizer(examples['text'], padding='longest', return_tensors='pt')
    input_ids = tokens['input_ids'].to(model.device)
    attention_mask = tokens['attention_mask'].to(model.device)
    
    with torch.no_grad():
        model_out = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
    
    last_token_indices = attention_mask.sum(dim=-1) - 1
    embeddings = model_out.hidden_states[-1][torch.arange(input_ids.size(0)), last_token_indices]
    
    return {'embeddings': embeddings}

train_embeddings = dataset.map(compute_embeddings, batched=True, batch_size=64)
train_embeddings.set_format('torch')

Map:   0%|          | 0/4096 [00:00<?, ? examples/s]

In [9]:
tokens = tokenizer(dataset['text'][0], padding='longest', return_tensors='pt')
input_ids = tokens.input_ids.to(model.device)
attention_mask = tokens.attention_mask.to(model.device)

with torch.no_grad():
    model_output = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)

In [28]:
torch.stack(model_output.hidden_states)[:,0,:,:].float().mean(1)

tensor([[-1.9035e-03,  3.4298e-03,  1.1937e-02,  ..., -6.0098e-03,
         -5.7379e-03, -4.3657e-03],
        [-1.2339e-02,  1.1494e-02, -1.5648e-02,  ...,  4.3412e-03,
         -1.2122e-02,  1.6042e-03],
        [ 1.9405e-03,  6.4132e-03, -1.8607e-02,  ...,  9.9847e-03,
         -1.8577e-02, -5.6446e-03],
        ...,
        [-1.3252e-01, -3.6015e-02, -4.6515e-03,  ..., -1.4342e-01,
          3.2569e-02,  9.9657e-02],
        [-1.3976e-01,  3.2463e-02,  1.0859e-01,  ..., -1.3966e-01,
          2.4561e-03,  1.0292e-01],
        [-4.1964e-01,  2.4690e+00,  1.1171e+00,  ..., -2.7177e+00,
         -2.7629e+00,  7.1740e-01]], device='mps:0')

In [26]:
torch.stack(model_output.hidden_states)[:,0,:,:].float().sum(1) / (torch.stack(model_output.hidden_states)[:,0,:,:].shape[1])

tensor([[-1.9035e-03,  3.4298e-03,  1.1937e-02,  ..., -6.0098e-03,
         -5.7379e-03, -4.3657e-03],
        [-1.2339e-02,  1.1494e-02, -1.5648e-02,  ...,  4.3412e-03,
         -1.2122e-02,  1.6042e-03],
        [ 1.9405e-03,  6.4132e-03, -1.8607e-02,  ...,  9.9847e-03,
         -1.8577e-02, -5.6446e-03],
        ...,
        [-1.3252e-01, -3.6015e-02, -4.6515e-03,  ..., -1.4342e-01,
          3.2569e-02,  9.9657e-02],
        [-1.3976e-01,  3.2463e-02,  1.0859e-01,  ..., -1.3966e-01,
          2.4561e-03,  1.0292e-01],
        [-4.1964e-01,  2.4690e+00,  1.1171e+00,  ..., -2.7177e+00,
         -2.7629e+00,  7.1740e-01]], device='mps:0')

In [76]:
# Just don't use batching?

In [94]:
def compute_embeddings(examples):
    tokens = tokenizer(examples['text'], padding='longest', return_tensors='pt')
    input_ids = tokens['input_ids'].to(model.device)
    attention_mask = tokens['attention_mask'].to(model.device)
    
    with torch.no_grad():
        model_out = model(input_ids=input_ids, output_hidden_states=True)

    # TODO: Does output_type do anything?
    output_type = model_out.hidden_states[0].dtype
    
    layer_token_embeddings = torch.stack(model_out.hidden_states)[:,0].float()

    layer_last_embeddings = layer_token_embeddings[-1]
    layer_mean_embeddings = layer_token_embeddings.mean(1)

    return {'layer_last_embeddings': layer_last_embeddings.to(output_type), 'layer_mean_embeddings': layer_mean_embeddings.to(output_type)}

train_embeddings3 = dataset.map(compute_embeddings)
train_embeddings3.set_format('torch')

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

In [ ]:
def compute_embeddings(examples):
    tokens = tokenizer(examples['text'], padding='longest', return_tensors='pt')
    input_ids = tokens['input_ids'].to(model.device)
    attention_mask = tokens['attention_mask'].to(model.device)
    
    with torch.no_grad():
        model_out = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)

    # TODO: Does output_type do anything?
    output_type = model_out.hidden_states[0].dtype
    
    input_layer_token_embeddings = torch.stack(model_out.hidden_states).permute(1,0,2,3).float()

    last_token_indices = attention_mask.sum(-1) - 1
    layer_last_embeddings = input_layer_token_embeddings[torch.arange(input_layer_token_embeddings.shape[0]), :, last_token_indices]
    layer_mean_embeddings = (input_layer_token_embeddings * attention_mask.unsqueeze(-1).unsqueeze(1)).sum(2) / attention_mask.sum(1).unsqueeze(-1).unsqueeze(-1)

    return {'layer_last_embeddings': layer_last_embeddings.to(output_type), 'layer_mean_embeddings': layer_mean_embeddings.to(output_type)}

train_embeddings2 = dataset.map(compute_embeddings, batched=True, batch_size=1)
train_embeddings2.set_format('torch')

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

In [33]:
torch.cuda.empty_cache()

In [95]:
train_embeddings3['layer_last_embeddings'][-1][-1]

tensor([-0.6445,  3.2969,  0.7656,  ..., -4.8750, -2.5000,  1.7969])

In [70]:
train_embeddings2['layer_last_embeddings'][-1,-1]

tensor([-0.6445,  3.2969,  0.7656,  ..., -4.8750, -2.5000,  1.7969])

In [96]:
train_embeddings3['layer_mean_embeddings'][-1][-1]

tensor([ 0.0425,  3.3438,  0.8633,  ..., -1.7344, -2.3438,  1.2422])

In [71]:
train_embeddings2['layer_mean_embeddings'][-1,-1]

tensor([ 0.0425,  3.3438,  0.8633,  ..., -1.7344, -2.3438,  1.2422])

In [88]:
with torch.no_grad():
    model_output = model(input_ids=tokenizer(train_embeddings3['text'][-1], padding='longest', return_tensors='pt').input_ids.to(model.device), output_hidden_states=True)

In [89]:
torch.stack(model_output.hidden_states)[-1,0,-1]

tensor([-0.6445,  3.2969,  0.7656,  ..., -4.8750, -2.5000,  1.7969],
       device='mps:0', dtype=torch.bfloat16)

In [93]:
torch.stack(model_output.hidden_states)[:,0].mean(1).shape

torch.Size([17, 2048])

In [90]:
torch.stack(model_output.hidden_states).mean(2)[-1,0]

tensor([ 0.0425,  3.3438,  0.8633,  ..., -1.7344, -2.3438,  1.2422],
       device='mps:0', dtype=torch.bfloat16)

In [98]:
(torch.stack(model_output.hidden_states).mean(2)[-1,0].cpu() - train_embeddings3['layer_mean_embeddings'][-1,-1]).abs().mean()

tensor(0.)

In [97]:
(torch.stack(model_output.hidden_states).mean(2)[-1,0].cpu() - train_embeddings2['layer_mean_embeddings'][-1,-1]).abs().mean()

tensor(0.)

In [75]:
(torch.stack(model_output.hidden_states).float().mean(2)[-1,0].cpu() - train_embeddings2['layer_mean_embeddings'][-1,-1]).abs().mean()

tensor(0.0019)

In [99]:
(torch.stack(model_output.hidden_states).float().mean(2)[-1,0].cpu() - train_embeddings3['layer_mean_embeddings'][-1,-1]).abs().mean()

tensor(0.0019)

In [141]:
input_layer_token_embeddings[0,-1,0]

tensor([ 1.5547, -1.5234,  2.7812,  ..., -0.6445,  0.6094, -1.0469],
       device='cuda:0', dtype=torch.bfloat16)

In [142]:
torch.stack(model_output.hidden_states).shape

torch.Size([17, 1, 98, 2048])

In [140]:
input_layer_token_embeddings = torch.stack(model_output.hidden_states).permute(1,0,2,3)

In [121]:
train_embeddings['layer_mean_embeddings'][-1] - torch.stack(model_output.hidden_states)[:,0].mean(1).cpu()

tensor([[ 0.0000e+00,  0.0000e+00, -6.1035e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  1.5259e-05,
          0.0000e+00,  0.0000e+00],
        [ 3.0518e-05,  1.5259e-05,  2.4414e-04,  ...,  0.0000e+00,
          0.0000e+00,  3.0518e-05],
        ...,
        [-9.7656e-04,  0.0000e+00,  4.8828e-04,  ..., -9.7656e-04,
         -4.8828e-04,  0.0000e+00],
        [ 0.0000e+00,  9.7656e-04, -1.2207e-04,  ...,  1.9531e-03,
         -1.2207e-03,  8.5449e-04],
        [-3.9062e-03,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  3.9062e-03]])

In [98]:
torch.stack(model_output.hidden_states).shape

torch.Size([17, 1, 161, 2048])

In [99]:
torch.stack(model_output.hidden_states)[:,0,-1]

tensor([[ 1.1414e-02, -2.2278e-03, -1.1292e-03,  ..., -1.7944e-02,
         -2.6855e-03, -2.1729e-02],
        [ 2.0142e-03,  4.8828e-03, -1.0400e-01,  ..., -2.1729e-02,
         -2.8809e-02, -3.7354e-02],
        [ 7.7515e-03, -5.6152e-02, -1.5430e-01,  ...,  5.4932e-02,
         -3.9551e-02, -6.4941e-02],
        ...,
        [-9.5703e-02,  3.2227e-01, -5.9766e-01,  ..., -7.6562e-01,
         -7.1289e-02, -5.2246e-02],
        [ 1.9141e-01,  4.3359e-01, -5.6641e-01,  ..., -6.3281e-01,
         -9.3262e-02, -6.0547e-02],
        [ 2.5312e+00,  3.9062e+00,  2.3730e-01,  ..., -4.7188e+00,
         -4.8438e+00, -1.0781e+00]], device='cuda:0', dtype=torch.bfloat16)

In [104]:
torch.stack(model_output.hidden_states)[:,0].mean(1)

tensor([[-9.9182e-04,  3.4027e-03,  1.2512e-02,  ..., -8.8501e-04,
         -6.8970e-03, -5.7068e-03],
        [-1.2085e-02,  8.4229e-03, -1.9653e-02,  ...,  9.2773e-03,
         -2.4048e-02,  4.2419e-03],
        [ 1.1292e-03,  3.9673e-03, -2.4658e-02,  ...,  1.7700e-02,
         -3.5400e-02, -3.6926e-03],
        ...,
        [ 3.4668e-02,  1.6309e-01, -1.3184e-01,  ..., -9.6680e-02,
         -1.6724e-02,  7.3730e-02],
        [ 9.0332e-02,  2.1484e-01, -7.2754e-02,  ..., -7.2754e-02,
         -1.3281e-01, -2.6611e-02],
        [ 1.3125e+00,  3.5312e+00,  6.1719e-01,  ..., -2.7969e+00,
         -3.2812e+00, -1.4648e-01]], device='cuda:0', dtype=torch.bfloat16)

In [8]:
def get_model_out(examples):
    tokens = tokenizer(examples['text'], padding='longest', return_tensors='pt')
    input_ids = tokens['input_ids'].to(model.device)
    attention_mask = tokens['attention_mask'].to(model.device)
    
    with torch.no_grad():
        model_out = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'model_out': model_out}

In [9]:
out = get_model_out(dataset.take(10))
torch.cuda.empty_cache()

In [11]:
input_ids = out['input_ids'].cpu()
attention_mask = out['attention_mask'].cpu()
model_out = out['model_out']

In [12]:
last_token_indices = attention_mask.sum(-1) - 1

In [23]:
torch.stack(model_out.hidden_states).cpu().permute(1,0,2,3).shape

torch.Size([10, 17, 452, 2048])

In [59]:
input_layer_token_embeddings = torch.stack(model_out.hidden_states).permute(1,0,2,3).cpu()

In [60]:
layer_last_embedding = input_layer_token_embeddings[torch.arange(input_layer_token_embeddings.shape[0]), :, last_token_indices]

In [42]:
input_layer_token_embeddings.shape

torch.Size([10, 17, 452, 2048])

In [50]:
attention_mask.shape

torch.Size([10, 452])

In [54]:
attention_mask.unsqueeze(0).shape

torch.Size([1, 10, 452])

In [73]:
attention_mask.unsqueeze(-1).unsqueeze(1).shape

torch.Size([10, 1, 452, 1])

In [80]:
attention_mask.sum(1)

tensor([315, 279, 146, 452, 160, 218, 361, 208, 177, 216])

In [83]:
((input_layer_token_embeddings * attention_mask.unsqueeze(-1).unsqueeze(1)).sum(2) / attention_mask.sum(1).unsqueeze(-1).unsqueeze(-1)).shape

torch.Size([10, 17, 2048])

In [84]:
layer_mean_embeddings = (input_layer_token_embeddings * attention_mask.unsqueeze(-1).unsqueeze(1)).sum(2) / attention_mask.sum(1).unsqueeze(-1).unsqueeze(-1)

In [85]:
layer_mean_embeddings

tensor([[[-1.9073e-03,  3.4180e-03,  1.1902e-02,  ..., -5.9814e-03,
          -5.7068e-03, -4.3640e-03],
         [-1.2329e-02,  1.1475e-02, -1.5625e-02,  ...,  4.3640e-03,
          -1.2085e-02,  1.6098e-03],
         [ 1.9531e-03,  6.3782e-03, -1.8555e-02,  ...,  1.0010e-02,
          -1.8433e-02, -5.5847e-03],
         ...,
         [-1.3086e-01, -3.5156e-02, -4.4861e-03,  ..., -1.4258e-01,
           3.2959e-02,  9.8633e-02],
         [-1.3965e-01,  3.3691e-02,  1.0840e-01,  ..., -1.3867e-01,
           2.4261e-03,  1.0107e-01],
         [-4.1406e-01,  2.4688e+00,  1.1172e+00,  ..., -2.7031e+00,
          -2.7656e+00,  7.1094e-01]],

        [[-2.7924e-03,  2.7618e-03,  1.0254e-02,  ..., -3.5400e-03,
          -4.1809e-03, -3.6621e-03],
         [-1.1230e-02,  1.0254e-02, -2.5635e-02,  ...,  3.9062e-03,
          -2.1729e-02,  2.5177e-03],
         [ 4.8065e-04,  8.8501e-03, -3.1006e-02,  ...,  1.0986e-02,
          -4.1504e-02, -8.0566e-03],
         ...,
         [ 4.1992e-02,  9

In [ ]:
(input_layer_token_embeddings.permute(1,0,2,3) * attention_mask.unsqueeze(-1)).mean().shape

torch.Size([17, 10, 452, 2048])

In [127]:
layer_mean_embeddings = ((torch.stack(model_out.hidden_states)[:, torch.arange(input_ids.size(0))] * attention_mask.unsqueeze(-1)).sum(2) / attention_mask.sum(-1).unsqueeze(-1)).permute(1,0,2)

torch.Size([10, 17, 2048])